# WebAssembly powered by wasmtime

In [1]:
#include <wasmtime/wasm.h>
#define own
#include <wasmtime/wasmtime.h>
#pragma cling load("libwasmtime")

In [2]:
wasm_engine_t* engine = wasm_engine_new()

In [3]:
engine

@0x7ffef27f3118

In [4]:
const char* addWAT = R"WAT((module
  (type (;0;) (func (param i32 i32) (result i32)))
  (func $add (type 0) (param i32 i32) (result i32)
    get_local 1
    get_local 0
    i32.add)
  (memory 1)
  (export "memory" (memory 0))
  (export "add" (func $add))
))WAT";

In [5]:
wasm_byte_vec_t wat;
wat.size = strlen(addWAT);
wat.data = (byte_t*)addWAT;

In [6]:
#include <iostream>

Convert WAT to WASM.

In [7]:
wasm_byte_vec_t wasm;
wasm.size = 0;
wasm_message_t errorMessage;
if (!wasmtime_wat2wasm(
     engine,
     &wat,
     &wasm,
     &errorMessage)) {
  std::cerr << "Failed." << std::endl;
}

In [8]:
wasm.size

70

Create store.

In [9]:
wasm_store_t* store = wasm_store_new(engine); store

@0x7ffef27f3118

Check if WASM is valid.

In [10]:
bool status = wasm_module_validate(store, &wasm); status

true

Create module.

In [11]:
wasm_module_t* module = wasm_module_new(store, &wasm); module

@0x7ffef27f3118

Get imports.

In [12]:
wasm_importtype_vec_t imports;
wasm_module_imports(module, &imports);
imports.size

0

Create instance.

In [13]:
own wasm_trap_t* trap = nullptr;
own wasm_instance_t* instance = wasm_instance_new(store, module, nullptr, &trap);
instance

@0x7ffef27f3118

Get exports.

In [14]:
own wasm_exporttype_vec_t exports;
wasm_module_exports(module, &exports);

In [15]:
exports.size

2

In [16]:
for (size_t i = 0; i < exports.size; ++i) {
  const wasm_exporttype_t* e = exports.data[i];
  const wasm_name_t* name = wasm_exporttype_name(e);
  std::cout << "Name = " << std::string((const char*)(name->data), name->size) << std::endl;
  const wasm_externtype_t* type = wasm_exporttype_type(e);
  // wasm_externkind_t kind = wasm_externtype_kind(type);
  // std::cout << "Extern kind = " << kind << std::endl;
}

Name = memory
Name = add
